In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dcase_evaluator_analysis import DCASEEvaluatorAnalysis
from models.audiosep import AudioSep
# from models.one_peace_encoder import ONE_PEACE_Encoder
from models.clap_encoder import CLAP_Encoder

import argparse
import os
from utils import parse_yaml, load_ss_model
import pandas as pd
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import librosa
import numpy as np


def eval(evaluator,
         encoder_checkpoint_path = None, 
         ssnet_checkpoint_path = None, 
         config_yaml=None, 
         device = "cuda"):
    
    configs = parse_yaml(config_yaml)
    
    # # ONE_PEACE modelhub expects some paths to be relative to this dir
    # os.chdir('ONE-PEACE/')
    # # TODO:path in shared scratch dir for now..., move to class project dir whenever we get that
    # query_encoder = ONE_PEACE_Encoder(pretrained_path=encoder_checkpoint_path)
    # os.chdir('..')

     # Load model
    query_encoder = CLAP_Encoder().eval()

    # pl_model = load_ss_model(
    #     configs=configs,
    #     checkpoint_path=encoder_checkpoint_path,
    #     query_encoder=query_encoder
    # ).to(device)

    # put ONE-PEACE model in eval model (probably unecessary)
    # query_encoder.model.model.eval()

    pl_model = load_ss_model(
        configs=configs,
        checkpoint_path=ssnet_checkpoint_path,
        query_encoder=query_encoder
    ).to(device)


    print(f'-------  Start Evaluation  -------')

    # evaluation 
    return evaluator(pl_model)
    msg_clotho = "SDR: {:.3f}, SDRi: {:.3f}, SISDR: {:.3f}".format(SDR, SDRi, SISDR)
    print(msg_clotho)

    print('-------------------------  Done  ---------------------------')



/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/nfshomes/vla/lass-final-project/models/CLAP/open_clip/factory.py:55: FutureWarning: You a

In [3]:

parser = argparse.ArgumentParser()
parser.add_argument(
    "--config_yaml",
    type=str,
    required=True,
    help="Path of config file for AudioSep model",
)

parser.add_argument(
    "--encoder_checkpoint_path",
    type=str,
    required=True,
    help="Path of pretrained checkpoint for QueryEncoder (ONE-PEACE)",
)

parser.add_argument(
    '--ssnet_checkpoint_path',
    type=str,
    required=True,
    help = "Path of pretrained checkpoint for Seperation Network (ResUNet)"
)

cli = '--config_yaml config/audiosep_base.yaml --encoder_checkpoint_path ./checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt  --ssnet_checkpoint_path checkpoint/audiosep_baseline.ckpt'

args = parser.parse_args(cli.split())
args



Namespace(config_yaml='config/audiosep_base.yaml', encoder_checkpoint_path='./checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt', ssnet_checkpoint_path='checkpoint/audiosep_baseline.ckpt')

In [ ]:
# # Run evaluation on validation set + pull out per-sample metrics and similarity scores
# dcase_evaluator = DCASEEvaluatorAnalysis(
#     sampling_rate=16000,
#     eval_indexes='lass_synthetic_validation.csv',
#     audio_dir='lass_validation',
#     output_dir = '/fs/nexus-scratch/vla/lass_validation_baseline_output',
#     encoder_type='CLAP'
# )


# Run evaluation on validation set + pull out per-sample metrics and similarity scores
dcase_evaluator = DCASEEvaluatorAnalysis(
    sampling_rate=16000,
    eval_indexes='lass_synthetic_validation.csv',
    audio_dir='lass_validation',
    output_dir = None,
    encoder_type='CLAP'
)



In [5]:
df_results = eval(dcase_evaluator,
                  encoder_checkpoint_path = args.encoder_checkpoint_path,
                  ssnet_checkpoint_path = args.ssnet_checkpoint_path,
                  config_yaml = args.config_yaml,
                  device = "cuda")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/lightning/pytorch/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['query_encoder.model.text_branch.embeddings.position_ids']


-------  Start Evaluation  -------
Evaluation on DCASE T9 synthetic validation set.


  0%|          | 0/3000 [00:00<?, ?it/s]/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(
100%|██████████| 3000/3000 [05:14<00:00,  9.54it/s]


In [6]:
df_results['delta_similarity'] = df_results['output_similarity'] - df_results['input_similarity']

In [8]:
df_results.sort_values(by='delta_similarity', ascending=False)[:5]

,caption,source_path,noise_path,input_path,output_path,input_similarity,output_similarity,target_similarity,sisdr,sdri,sdr,delta_similarity
621,There is a sound of explosion in the distance.,lass_validation/699245_15101941-hq.wav,lass_validation/686774_11771918-hq.wav,/fs/nexus-scratch/vla/lass_validation_baseline...,/fs/nexus-scratch/vla/lass_validation_baseline...,-0.077745,0.331277,0.450373,1.711304,13.812068,3.812069,0.409022
996,"The roaring sound of a thunder is really loud,...",lass_validation/686579_1089955-hq.wav,lass_validation/698177_7554526-hq.wav,/fs/nexus-scratch/vla/lass_validation_baseline...,/fs/nexus-scratch/vla/lass_validation_baseline...,0.137888,0.508465,0.505981,19.711496,21.666628,19.666626,0.370577
2890,"A person is taking a deep breath, and the soun...",lass_validation/706271_2618335-hq.wav,lass_validation/686610_13228046-hq.wav,/fs/nexus-scratch/vla/lass_validation_baseline...,/fs/nexus-scratch/vla/lass_validation_baseline...,-0.075675,0.290597,0.408750,-1.698891,10.739242,1.739242,0.366272
774,A person is drawing the cork and then it comes...,lass_validation/707290_1089955-hq.wav,lass_validation/696314_1089955-hq.wav,/fs/nexus-scratch/vla/lass_validation_baseline...,/fs/nexus-scratch/vla/lass_validation_baseline...,0.112191,0.477732,0.257591,-0.395025,6.817269,2.817269,0.365541
2144,"A door creaks open, keys jangle, and the sound...",lass_validation/695948_15107438-hq.wav,lass_validation/706254_11686912-hq.wav,/fs/nexus-scratch/vla/lass_validation_baseline...,/fs/nexus-scratch/vla/lass_validation_baseline...,0.061771,0.425480,0.433587,5.871135,19.742934,6.742934,0.363708


In [10]:
df_results.to_csv('results/baseline_validation_results.csv', index=None)